In [1]:
import openai
from importlib import reload
from utils.structure_content import process_folder_concurrently,StructuredDocument
from llm_client import llm_client
from llm_client.agent import MultiTurnAgent
from llm_client.prompt_builder import  PromptBuilder
from llm_client.document_vector_store import (
    EmbeddingProcessor,
    Document,
    DocumentStore,
    SimpleDocument,
    VectorStore
)
from kme_doc import KMEDocument
from tqdm import tqdm
import pandas as pd
import os
from content_retrieval import query_document, llm,doc_store, summary_doc_store ,embed,embedding
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore
from config.settings import Settings
settings = Settings()

2025-09-10 11:03:18,997 - INFO - Loading faiss with AVX512 support.
2025-09-10 11:03:18,998 - INFO - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-09-10 11:03:18,998 - INFO - Loading faiss with AVX2 support.
2025-09-10 11:03:19,010 - INFO - Successfully loaded faiss with AVX2 support.
2025-09-10 11:03:19,931 - INFO - HTTP Request: POST http://127.0.0.1:1234/v1/embeddings "HTTP/1.1 200 OK"


Initialized new FAISS index with dimension 2560.
Syncing VectorStore with DocumentStore...
Found 358 missing documents. Indexing in batches of 128...


2025-09-10 11:03:25,632 - INFO - HTTP Request: POST http://127.0.0.1:1234/v1/embeddings "HTTP/1.1 200 OK"


Saved FAISS index (128 vectors) and ID set.


2025-09-10 11:03:31,223 - INFO - HTTP Request: POST http://127.0.0.1:1234/v1/embeddings "HTTP/1.1 200 OK"


Saved FAISS index (256 vectors) and ID set.


2025-09-10 11:03:35,514 - INFO - HTTP Request: POST http://127.0.0.1:1234/v1/embeddings "HTTP/1.1 200 OK"


Saved FAISS index (358 vectors) and ID set.
Saved FAISS index (358 vectors) and ID set.
Saved FAISS index (358 vectors) and ID set.
Sync complete.


In [2]:
kme_vertaaltabel = pd.read_csv(settings.data_root /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

# Opbouwen Doc store van pdf

In [3]:
kme_documents = pd.read_pickle(settings.content_folder / "extracted_df_km.pkl").reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()
ob_documents = kme_documents.query("BELASTINGSOORT == 'OB - Omzetbelasting'")
results = add_new_documents_to_docstore(ob_documents,doc_store)

In [4]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')

### Samenvatten en segmenteren content

In [5]:
stats = summarize_new_documents(
    doc_store=doc_store,
    summary_doc_store=summary_doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    KMEDocument_cls=KMEDocument,
    max_workers=10,
    start=0,
    count=1500,
    show_progress=True
)

Summarizing: 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]


## Testing Summary

In [23]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')
from content_retrieval import query_document, llm,doc_store, summary_doc_store ,embed,embedding
from implementations.tools import VectorSearchTool, DocumentSearchTool, DocumentShortlistTool
vs_tool = VectorSearchTool(vector_store=embedding)
slt = DocumentShortlistTool()
prompt_processor = PromptBuilder('prompt_templates','search') 
agent = MultiTurnAgent(
    llm_processor=llm,
    prompt_processor=prompt_processor,
    tools=[vs_tool,slt] # Pass the instantiated tool(s)
)

In [24]:
doc = doc_store.documents['KM1001954']

In [25]:
prompt = summary_processor.create_prompt(document = doc.content,question= doc.metadata['VRAAG'],
                                        taxonomy_path = {'Belastingsoort': doc.metadata['BELASTINGSOORT'], 
                                                        'Proces': doc.metadata['PROCES_ONDERWERP'],
                                                        'Product' : doc.metadata['PRODUCT_SUBONDERWERP']})

In [26]:
results = llm.process(prompt)

2025-09-10 10:52:51,345 - INFO - HTTP Request: POST https://ino-ai-foundry-kis.openai.azure.com/openai/deployments/gpt-oss-120b/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"


In [27]:
results.content

{'content': 'Dit document gaat over Omzetbelasting (OB/BTW) – proces Aangifte met betrekking tot Aangifte. Het beschrijft de mogelijkheden en termijnen voor het indienen van correcties en suppleties voor zowel lopende als geopende ondernemingen. Antwoord: Correcties en suppleties kunnen worden ingediend tot en met vijf (5) jaar na het kalenderjaar waarop de correctie of suppletie betrekking heeft. Deze termijn geldt eveneens voor een onderneming die inmiddels is afgevoerd. Een suppletie kan per aangiftetijdvak worden ingediend; wanneer de suppletie voor het volledige jaar wordt ingediend, hoeven de gewijzigde gegevens niet per aangiftetijdvak te worden opgesplitst, maar dienen ze als een geheel voor het betreffende jaar te worden genoemd. Geen Interne informatie aanwezig.',
 'metadata': {'Tags': ['Omzetbelasting',
   'Aangifte',
   'Suppletie',
   'Correctie',
   'Terugwerkende termijn',
   'Aangiftetijdvak',
   'Afgevoerde onderneming',
   'Terugwerkingsperiode']}}